Demo for the new plotting functions of `BioPsyKit`: `bp.plotting.feature_boxplot()` and `bp.plotting.multi_feature_boxplot()` and the derived functions: `bp.protocols.plotting.saliva_feature_boxplot()` and `bp.protocols.plotting.saliva_multi_feature_boxplot()`.

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.protocols import CAR

from carwatch_analysis.saliva_helper import import_cortisol_raw, import_cortisol_features, analysis_saliva_raw, analysis_saliva_features
from carwatch_analysis.general_helper import describe_groups_df

from statannot import add_stat_annotation

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
pg.options['round'] = 3

sns.set(style='ticks', context='talk')
#plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['figure.figsize'] = (10,5)
plt.close('all')

export = True

palette = bp.colors.fau_palette
sns.set_palette(palette)
palette

In [ ]:
export_path = Path("../../../exports")
plot_path = export_path.joinpath("plots")
stats_path = export_path.joinpath("statistics")

paper_path = Path("/Users/Richer/Documents/PhD/Research/Papers/2021_CARInnerClock_EMBC/img")
#paper_path = paper_path.joinpath('img')

bp.utils.file_handling.mkdirs([plot_path, stats_path, paper_path])

## Data Import

In [ ]:
car = CAR(saliva_times=[i*15 for i in range(5)])

### Raw Cortisol

In [ ]:
cort_samples = import_cortisol_raw(export_path.joinpath("cortisol_samples_cleaned.csv"))
cort_samples.head()

### Cortsol Features

In [ ]:
cort_features = import_cortisol_features(export_path.joinpath("cortisol_features_cleaned.csv"))
cort_features.head()

In [ ]:
pipeline = analysis_saliva_features(cort_features, variable="condition", test_type='anova')
pipeline.display_results(posthoc=True, sig_only="posthoc")
if export:
    pipeline.export_statistics(stats_path.joinpath("analysis_cortisol_features_condition.xlsx"))

In [ ]:
feature = 'auc_i'

box_pairs, pvals = pipeline.sig_brackets(pipeline.results_cat('posthoc'), stats_type='between', plot_type='single', features=feature)
data_plot = cort_features.xs(feature, level='biomarker')

fig, ax = plt.subplots()
car.saliva_feature_boxplot(data=data_plot, x='condition', saliva_type='cortisol', feature=feature, order=order, stats_kwargs={'box_pairs': box_pairs, 'pvalues': pvals}, ax=ax)
fig.tight_layout()

In [ ]:
feature = 'slopeS0S4'

box_pairs, pvals = pipeline.sig_brackets(pipeline.results_cat('posthoc'), stats_type='between', plot_type='multi', features=feature)

data = cort_features.xs('slopeS0S4', level='biomarker', drop_level=False)

fig, ax = plt.subplots()
car.saliva_feature_boxplot(data=data, x='biomarker', saliva_type='cortisol', hue='condition', feature=feature, hue_order=order, stats_kwargs={'box_pairs': box_pairs, 'pvalues': pvals}, ax=ax)
fig.tight_layout()

In [ ]:
features = ['slopeS0S3', 'slopeS0S4']

box_pairs, pvals = pipeline.sig_brackets(pipeline.results_cat('posthoc'), stats_type='between', plot_type='multi', features=features)

data = cort_features.filter(like='slope', axis=0)

fig, ax = plt.subplots()
car.saliva_feature_boxplot(data=data, x='biomarker', saliva_type='cortisol', hue='condition', hue_order=order, feature=features, stats_kwargs={'box_pairs': box_pairs, 'pvalues': pvals}, ax=ax)
fig.tight_layout()